In [1]:
cd /home/urwa/Documents/side_projects/urban/UrbanTrafficPrediction/

/home/urwa/Documents/side_projects/urban/UrbanTrafficPrediction


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
from utils.lstm_utils import prepare_data_lstm, lstm_temporal_dataloader
from utils.lstm_utils import get_device

from utils.lstm_utils import evaluate_edge, load_edge_test_data, get_edge_pred_df
from models.models import LSTM

from utils.lstm_utils import train_one_epoch, store_chekpoint, evaluate_lstm_pipeline_model, run_inference

In [4]:
torch.manual_seed(2020)
np.random.seed(2020)

In [5]:
config = {'lr': 0.00034439316653688684,
 'layers': 3,
 'step_size': 11,
 'gamma': 0.761795969995615,
 'bptt': 19,
 'dropout': 0.1227497445640586}

In [6]:
device = get_device(cuda=True)
device

device(type='cuda')

In [ ]:
data_path = '/home/urwa/Documents/side_projects/urban/data/featureData/com_jfk.csv'
weights_path = '/home/urwa/Documents/side_projects/urban/data/featureData/com_jfk_weights.csv'
test_data_path='/home/urwa/Documents/side_projects/urban/data/featureData/jfk.csv'

In [7]:
dataset, targetColumns, features_cols = prepare_data_lstm(data_path)

Raw Shape:  (8757, 113)
Cleaned Shape:  (8757, 38)
Target columns: 24
Feature coumns:  13


In [8]:
bptt = config['bptt']
train_ratio = 0.75

In [9]:
train_inout_seq,test_inout_seq = lstm_temporal_dataloader(dataset, features_cols, targetColumns, 
                                                          train_ratio, bptt, device)

train test split
train shape:  (6567, 38)
test shape:  (2190, 38)
train feature tensor shape : torch.Size([6567, 13])
train target tensor shape : torch.Size([6567, 24])
test feature tensor shape : torch.Size([2190, 13])
test target tensor shape : torch.Size([2190, 24])

sequences
torch.Size([19, 13]) torch.Size([19, 24]) torch.Size([19, 24])


In [12]:
lstm_layers = config['layers']
network_size = len(targetColumns)
feat_size = len(features_cols)
dropout = config['dropout']
hidden_layer_size=100

# layers = 2
model = LSTM(feat_size, network_size, hidden_layer_size, lstm_layers, dropout).to(device)
loss_function = nn.L1Loss()   
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=config['step_size'], gamma=config['gamma'])


In [13]:
exp_dir = 'data'
epochs = 1
for i in range(epochs):
    loss = train_one_epoch(model, optimizer, loss_function, train_inout_seq, device)
    
    scheduler.step()
    store_chekpoint(exp_dir, model, optimizer, scheduler)
    residual, r2, rmse, mae = evaluate_lstm_pipeline_model(model, test_inout_seq, device)
    print(f'epoch: {i:3} loss: {loss:10.8f} r2: {r2:5.3f} rmse: {rmse:5.3f} mae: {mae:5.3f}')

------- Saving checkpoint---------------
epoch:   0 loss: 9.61490517 r2: 0.549 rmse: 291.687 mae: 8.170


In [14]:
y_pred_ = run_inference(model, test_inout_seq, device)
y_pred_.shape

(2171, 24)

In [19]:
edge_prediction_df = get_edge_pred_df(y_pred_, targetColumns, weights_path)
edge_prediction_df.shape

(2171, 258)

In [20]:
edge_testData = load_edge_test_data(test_data_path, train_ratio, bptt)
edge_testData.shape

(2171, 258)

In [21]:
res, r2, rmse, mae = evaluate_edge(edge_testData, edge_prediction_df)
r2

0.37094737744152406